<a href="https://colab.research.google.com/github/mvm1964/MGVB/blob/main/ann_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [29]:
# prompt: create a torch neural net class with two hided layers and a single continuous node in the output layer

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, 1) # Single continuous output node
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [30]:
# prompt: create pandas dataframe with 20 continuous columns and 1000 rows filled with random numbers from 0 to 1

import pandas as pd
import numpy as np

# Create a DataFrame with 20 continuous columns and 1000 rows
data = np.random.rand(1000, 20)
df = pd.DataFrame(data)

df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.471033,0.720309,0.476492,0.612154,0.190136,0.549185,0.662156,0.901717,0.541046,0.031873,0.985589,0.882697,0.261708,0.690625,0.686616,0.904498,0.823023,0.742449,0.937399,0.037557
1,0.787396,0.234662,0.676662,0.532179,0.564046,0.160797,0.856212,0.570210,0.284962,0.763774,0.861525,0.889026,0.094293,0.456626,0.518349,0.497730,0.172980,0.934633,0.589006,0.261170
2,0.765685,0.863510,0.516057,0.518918,0.578137,0.578496,0.843968,0.804433,0.651613,0.667682,0.102281,0.139030,0.872991,0.059697,0.967171,0.556027,0.550599,0.506003,0.419003,0.952102
3,0.497315,0.498392,0.262298,0.049086,0.132447,0.750781,0.830484,0.013024,0.883721,0.243769,0.461965,0.821921,0.722378,0.514823,0.354974,0.609114,0.767775,0.074608,0.443019,0.481395
4,0.467895,0.851575,0.485955,0.481815,0.487249,0.968760,0.934765,0.902229,0.535418,0.560838,0.065810,0.581626,0.144422,0.580634,0.470978,0.351054,0.899747,0.004776,0.972842,0.530393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.636261,0.384188,0.277582,0.582833,0.305483,0.508265,0.802770,0.968291,0.940001,0.223132,0.013475,0.988359,0.088095,0.894969,0.599354,0.429667,0.784508,0.996654,0.792480,0.321115
996,0.800777,0.778623,0.453351,0.534536,0.608614,0.553407,0.167482,0.319773,0.917410,0.801004,0.488474,0.427369,0.387464,0.490947,0.291777,0.584999,0.872321,0.823649,0.030490,0.248539
997,0.849299,0.798972,0.688781,0.963433,0.511021,0.301641,0.988854,0.477466,0.154099,0.466288,0.344168,0.216812,0.498393,0.755617,0.278860,0.323631,0.067828,0.939401,0.845378,0.490680
998,0.696340,0.580547,0.010393,0.704267,0.780739,0.123369,0.183631,0.820769,0.374937,0.173819,0.399475,0.717253,0.387966,0.714046,0.930091,0.038040,0.423075,0.115183,0.611031,0.836948


In [32]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using {device} device")

Using cpu device


In [33]:
model = NeuralNetwork(input_size=19, hidden_size1=10, hidden_size2=10).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=19, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [34]:
X = torch.rand(1, 19, device=device)
model(X)

tensor([[-0.0030]], grad_fn=<AddmmBackward0>)

In [35]:
# Read data, convert to NumPy arrays
X = df.iloc[:, 1:20].values
y = df.iloc[:, 0].values

device = torch.device("cpu")

# convert into PyTorch tensors
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device).reshape(-1, 1)

# create DataLoader, then take one batch
loader = DataLoader(list(zip(X,y)), shuffle=True, batch_size=16)
for X_batch, y_batch in loader:
    print(X_batch, y_batch)
    break


tensor([[0.4487, 0.5894, 0.6377, 0.7524, 0.5849, 0.7145, 0.7155, 0.5076, 0.0179,
         0.3894, 0.4680, 0.4138, 0.0713, 0.2158, 0.8542, 0.0445, 0.1518, 0.4059,
         0.5974],
        [0.0745, 0.7683, 0.9987, 0.2640, 0.6527, 0.7682, 0.5645, 0.8629, 0.2287,
         0.8832, 0.3365, 0.7745, 0.6879, 0.3644, 0.2184, 0.8594, 0.5711, 0.7882,
         0.7938],
        [0.8455, 0.7658, 0.7141, 0.0913, 0.7605, 0.5897, 0.6610, 0.1023, 0.9874,
         0.4360, 0.0752, 0.0412, 0.7496, 0.8704, 0.9331, 0.7974, 0.6215, 0.8184,
         0.6582],
        [0.0725, 0.5108, 0.3498, 0.4572, 0.6462, 0.4075, 0.1037, 0.6854, 0.8459,
         0.5085, 0.0016, 0.7984, 0.9995, 0.6566, 0.2838, 0.4744, 0.1584, 0.2755,
         0.2682],
        [0.7869, 0.5008, 0.3166, 0.8495, 0.8094, 0.9751, 0.0195, 0.3911, 0.2782,
         0.3756, 0.5629, 0.8061, 0.6717, 0.1197, 0.1016, 0.5380, 0.4299, 0.2613,
         0.2581],
        [0.1244, 0.9686, 0.4834, 0.0364, 0.6528, 0.8285, 0.4878, 0.8591, 0.8425,
         0.9013, 0.

In [36]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train-test split for evaluation of the model
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

# set up DataLoader for training set
loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)


In [37]:
# train
n_epochs = 200
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# evaluate accuracy after training
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

Model accuracy: 0.00%
